# Predicting hotspots of burglaries

In this tutorial, hotspots of burglaries will be predicted in the city of Dallas. Three different models will be trained and their predictions will be validated.

Start by importing the hotspot library.

In [1]:
from datetime import date
import predictivehp.processing.data_processing as dp
import predictivehp.models.models as models
import predictivehp.visualization.plotter as pltr

%config InlineBackend.figure_format = 'retina'

## Importing crime data

The tutorial will consider crime data provided by the City of Dallas. The data can be imported with the Socrata API. The street map and city limits can be obtained from Dallas City Hall (https://gis.dallascityhall.com/shapefileDownload.aspx). For convenience, the data has already been stored in the package.

In [2]:
# CONSULTAR

# data = hp.import.tutorial('Dallas2017')
# The 'data' output is an instance of an object, which has all the relevant
# information and data such as a dataframe with incidences and shape files of
# the city. Either store the data in a folder or request data from Socrata with
# default parameters.

In general, data can be retrieved from the Socrata API and shape files can be specified as well.

In [3]:
b_path = 'predictivehp/data'
s_shp_p = f'{b_path}/streets.shp'
c_shp_p = f'{b_path}/councils.shp'
cl_shp_p = f'{b_path}/citylimit.shp'

pp = dp.PreProcessing()
shps = pp.shps_processing(s_shp_p, c_shp_p, cl_shp_p)
data = pp.get_data(year=2017, n=150000)

The crime data is stored in a Pandas DataFrame where columns **x** and **y** are the location coordinates in meters and **date** the date of the crime event.

In [4]:
data.head()

,x,y,date,month1,y_day
0,2.526789e+06,6.972383e+06,2017-01-01,January,1
1,2.525848e+06,6.971430e+06,2017-01-01,January,1
2,2.483264e+06,6.927857e+06,2017-01-01,January,1
3,2.504030e+06,6.966014e+06,2017-01-01,January,1
4,2.479656e+06,6.956554e+06,2017-01-01,January,1


## Specify the prediction configuration

Data has been loaded for the entire year of 2017. In this tutorial, the first week of November will be predicted from previous crime data. Since data is available for this week, the prediction can be validated as well.

All three available models, that is, STKDE, ProMap and Random Forest will be used for the prediction.

In [5]:
model = models.create_model(
    data=data, shps=shps,
    start_prediction=date(2017, 11, 1), length_prediction=7,
    use_stkde=False, use_promap=True, use_rfr=True,
)
# The output is an object that has all relevant attributes, for example the data
# for the incidences and a separate object for each model (stkde, promap, rfr).

## Specifying model parameters

Optionally, hyperparameters for the predictive models can be specified.

In [6]:
# model.stkde.set_parameters(bw=[1900, 2500, 38])

In [7]:
model.promap.set_parameters(bw=[1500, 1100, 35], hx=100, hy=100)

In [8]:
model.rfr.set_parameters(t_history=4,
                         xc_size=100, yc_size=100, n_layers=7,
                         label_weights=None,
                         read_data=True, read_X=True)

In [9]:
model.print_parameters()

ProMap Hyperparameters
bandwith x: 1500 mts
bandwith y: 1100 mts
bandwith t: 35 days
hx: 100 mts
hy: 100 mts

RFR Hyperparameters
Training History:   4 weeks
xc_size:            100 m
yc_size:            100 m
n_layers:           7
l_weights:          None



In [10]:
model.preprocessing()

## Training the models

The models will be trained on historical crime data to optimize the prediction.

In [11]:
model.fit()


Generating dataframe...

	Creating dataframe columns...
	Filling df...
		2017-10-04...  finished!
		2017-10-11...  finished!
		2017-10-18...  finished!
		2017-10-25...  finished!
		2017-11-01...  finished!
	Preparing df for filtering...
	Filtering cells...
		Loading shapefile...
		Creating GeoDataframe...
		Filtering...
		Updating dataframe... finished! 

The STKDE model has calculated the optimal space-time bandwidth for the prediction.

In [ ]:
model.stkde.bw
# Output is a list of the x-, y-, and t-bandwidth.

The ProMap model is a *lazy learner* and does not require any training: the space-time weights are fixed and only a rectangular mesh needs to be created.

The RFR model has created a forest of decision trees. Interestingly, the relative importance of the features can be retrieved. The table shows the feature importance with respect to the units of history and the depth of the layers.

In [ ]:
# model.rfr.plot_feature_importance()
# Output is a table with the index of the layer and the index of the histor
# unit (for example, week -1, week -2, until week -n). Color and values indicate
# the feature importance.

## Predicting future crimes

The crimes in the first week of November will be predicted with the different models. The results are risk scores that indicate the likelihood of a future crime to occur at the specific location. These propensity scores are normalized to a scale of 0 to 1.

In [12]:
model.predict()
# No output other than messages. Perform the calculations for the prediction.

The STKDE has created a function stating the risk associated to an arbitrary point on the map for any time in the prediction window. This score can be calculated at each point in space and time.

In [ ]:
model.stkde.score(x=123456, y=987654, t=4)
# Output is the value of the 'pdf' function in this point.

The ProMap model has created propensity scores for each cell in the rectangular mesh. These are stored in a matrix where each element corresponds to the score of the corresponding cell in the mesh.

In [ ]:
model.promap.score()
# Output is the matrix with scores.

Similarly, RFR also creates scores for each cell in a rectangular mesh.

In [ ]:
model.rfr.score().head()
# Output is the column of the dataframe with the predicted values.

## Visualizing the prediction

The prediction can be vizualized as a heat map, where the values indicate the likelihood of crime occurrence in the prediction window. Normally, hotspots of areas with a high crime risk can be seen.

In [13]:
models = [m for m in [model.stkde, model.promap, model.rfr] if m is not None]
plotter = pltr.Plotter(models=models,
                       n=100)
plotter.heatmap()

# TODO
# - Proyectar STKDE a metros
# - Estandarizar Títulos
# - Estandarizar Colorbar
# - Estandarizar facecolor de la fig

# Plot the heatmaps of all models, with the same axis and the same colorbar.
# Include optional parameters like 'crange=[0.2,0.8]' to plot values between 0.2
# and 0.8 only, cmap='jet', xlim=[123456,234567], savefig=True, etc.

AttributeError: 'NoneType' object has no attribute 'heatmap'

The STKDE method provides space-time prediction, which can be visualized with a four-dimensional plot.

In [ ]:
# model.stkde.plot_4Dheatmap()
# Output is the 4D heatmap. If not inline, store the data in a file.

## Validation of the prediction

When data is available for the prediction window, the predictions can be validated.

The testing procedures consists of an experiment where we assume that all areas with a high score will be patrolled and all incidences of crimes in this hotspot area will be detected. Multiple thresholds of the propensity score can be specified at the same time.

In [ ]:
# model.validate(c=[0.5, 0.9])
# No output other than messages. Perform calculations of the propensity score
# for these values of c.

The models have calculated the number of detected incidences for each score threshold.

In [ ]:
# model.detected_incidences()
# The output is the number of hits for each model and each value of c.

More detected incidences indicates a better performance of the model. However, this could be at the expense of a larger area to be patrolled. Hence, it is important to calculate the area of the hotspots as well.

In [ ]:
# model.hotspot_area()
# The output is the hotspot area for each model and each value of c.

### Hit rate

Different models can perform better for a smaller or larger area of the hotspots. Therefore, let us plot the hit rate with respect to the area. The *hit rate* is defined by the number of detected incidences inside the hotspot, divided by the total number of crime incidences in the prediction window.

In [14]:
plotter.hr()
# The output is a figure with the HR for each model.
# Optional parameters can be colors=['r','b','g'], models=['stkde','promap'],
# title='mytitle', savefig=True, etc.

AttributeError: 'NoneType' object has no attribute 'calculate_hr'

### Predictive accuracy index

The hit rate will always increase for larger hotspot areas. To balance the merits of many hits and the costs of a large area, the predictive accuracy index (PAI) can be used. It is defined as the hit rate divided by the percentage of the hotspot area compared to the city size.

In [ ]:
plotter.pai()
# The output is a figure with the PAI for each model.
# Optional parameters as in plot_hr().

### Heatmap

The performance can also be visualized by a heatmap. For a given score threshold, the hotspots are marked by the blue areas. The markers represent the incidences in the prediction window where green marker are hits and red markers misses.

In [ ]:
model.plot_heatmap(c=0.5, incidences=True)


# Marker rojo: inc dentro del hotspot
# Marker azul: inc fuera del hp


# Plot heatmaps for each model. In this case with red (or another color) for
# areas with c larger than the input.
# The rest of the area is transparent. Also, plot the incidences in the
# prediction window with markers (points, crosses, etc.) with different colors
# for the hits and misses.
# In general, you want to specify the area, not the c. For example,
# when area=0.05, you want to plot the hotspots with 5% of the total area. So,
# you need to retrieve the corresponding c for each model.

## Storing data

The data of the models can be stored, which is convenient to reduce computational time when performing the same simulation another time.

In [ ]:
model.store(file_name='model.data')
# Store all usefull data to disk. Can be NumPy arrays or Pickle objects and in
# different files.